In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install geopy tqdm openpyxl

In [ ]:
# --- 0. Asenna tarvittavat paketit (jos ei ole asennettuna) ---
# !pip install pandas requests geopy tqdm openpyxl

# --- 1. Tuodaan kirjastot ---
import requests
import pandas as pd
import numpy as np
import time
from tqdm.auto import tqdm
from geopy.geocoders import Nominatim
import os

# --- 2. ASETUKSET ---
BASE_URL     = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL     = f"{BASE_URL}/companies"
HEADERS      = {"Accept": "application/json"}

# Käytämme “postCode”‐parametria: '04600' kattaa Mäntsälän keskusta‐alueen osoitteet
MANTSALA_POSTCODE = "04600"
MAX_RESULTS       = 100  # Vain yksi haku, sillä Mäntsälässä alle 100 osoitetta

# Halutut TOL‐prefixit (47=kauppa, 62=ohjelmistot, 63=datan käsittely, 70=johtaminen/konsultointi, 73=markkinointi)
TARGET_TOLS_PREFIXES = ("47", "62", "63", "70", "73")

# Digitransit‐reitityksen asetukset
HOME_COORDS  = {"lat": 60.6320, "lon": 25.3187}  # Mäntsälän asema
GRAPHQL_URL  = "https://api.digitransit.fi/routing/v1/routers/hsl/index/graphql"
USER_AGENT   = "tyomatka_skanneri_demo/1.0 (oma@email)"

# --- 3. Hae kaikki yritykset, joiden osoiterekisterissä postCode = 04600 ---
print(f"Haetaan yritykset, joilla postinumero = {MANTSALA_POSTCODE}…")
params = {
    "postCode":   MANTSALA_POSTCODE,
    "maxResults": MAX_RESULTS,
    "resultsFrom": 0
}
r = requests.get(COMP_URL, params=params, headers=HEADERS, timeout=15)
r.raise_for_status()
data = r.json()
companies = data.get("companies", [])
print(f"Yrityksiä, joilla osoiterivi postinumero {MANTSALA_POSTCODE}: {len(companies)} riviä")

# --- 4. Muunna DataFrameksi ja pura TOL‐koodi (mainBusinessLine.type) ---
df = pd.json_normalize(companies)
df["tol_code"] = df["mainBusinessLine.type"].astype(str).fillna("")

# --- 5. Suodata TOL‐prefixien mukaan ---
mask_tol = df["tol_code"].apply(lambda x: any(x.startswith(prefix) for prefix in TARGET_TOLS_PREFIXES))
df_tol = df.loc[mask_tol].copy()
print(f"Suodatuksen jälkeen {len(df_tol)} riviä (TOL‐prefix + postinumero 04600).")

# --- 6. Pura yrityksen nimi (ensimmäinen elementti names‐listasta) ---
def get_company_name(name_list):
    if isinstance(name_list, list) and name_list:
        return name_list[0].get("name", None)
    return None

df_tol["company_name"] = df_tol["names"].apply(get_company_name)

# --- 7. Pura osoitetiedot (street, postcode, postOffice) kentästä 'addresses' ---
def extract_address_fields(addr_list):
    """
    Purkaa osoitelistan ensimmäisen kelvollisen dictionaryn, palauttaa:
      - 'street'
      - 'postcode'
      - 'postOffice'   (city)
      - 'municipality'
    """
    if not isinstance(addr_list, list) or not addr_list:
        return {"street": None, "postcode": None, "postOffice": None, "municipality": None}
    for addr in addr_list:
        if not isinstance(addr, dict):
            continue
        street = addr.get("street", None)
        postcode = addr.get("postCode", None)
        post_offices = addr.get("postOffices", [])
        post_office = None
        municipality = None
        if isinstance(post_offices, list) and post_offices:
            po = post_offices[0]
            post_office = po.get("city", None)
            municipality = po.get("city", None)
        # Jos katu ja postOffice löytyvät, palautetaan
        if street and postcode and post_office:
            return {
                "street":       street,
                "postcode":     postcode,
                "postOffice":   post_office,
                "municipality": municipality
            }
    # Jos ei löytynyt täydellistä, ota ensimmäinen
    first = addr_list[0]
    if isinstance(first, dict):
        street = first.get("street", None)
        postcode = first.get("postCode", None)
        post_offices = first.get("postOffices", [])
        post_office = None
        municipality = None
        if isinstance(post_offices, list) and post_offices:
            po = post_offices[0]
            post_office = po.get("city", None)
            municipality = po.get("city", None)
        return {
            "street":       street,
            "postcode":     postcode,
            "postOffice":   post_office,
            "municipality": municipality
        }
    return {"street": None, "postcode": None, "postOffice": None, "municipality": None}

df_tol["street"]       = df_tol["addresses"].apply(lambda L: extract_address_fields(L)["street"])
df_tol["postcode"]     = df_tol["addresses"].apply(lambda L: extract_address_fields(L)["postcode"])
df_tol["postOffice"]   = df_tol["addresses"].apply(lambda L: extract_address_fields(L)["postOffice"])
df_tol["municipality"] = df_tol["addresses"].apply(lambda L: extract_address_fields(L)["municipality"])

# Korvaa NaN tyhjällä merkkijonolla
for col in ["street", "postcode", "postOffice", "municipality"]:
    df_tol[col] = df_tol[col].fillna("")

print("\nSuodatetut rivit osoitetiedoilla (enintään 5 riviä):")
print(df_tol[["company_name", "street", "postcode", "postOffice"]].head(5))

# --- 8. Poista ne rivit, joilla street on tyhjä (tarvitaan geokoodaukseen) ---
df_with_street = df_tol[df_tol["street"] != ""].copy()
print(f"\nJäljelle katuosoitteiden perusteella: {len(df_with_street)} riviä")
print(df_with_street[["company_name", "street", "postcode", "postOffice"]])

# --- 9. Luo 'full_address' geokoodaukselle ---
df_with_street["full_address"] = (
    df_with_street["street"] + " " +
    df_with_street["postcode"].astype(str) + " " +
    df_with_street["postOffice"] + ", Mäntsälä"
)

# --- 10. GEOKOODAA välimuistin avulla ---
CACHE_FILE = "geocache.csv"
if os.path.exists(CACHE_FILE):
    cache_df = pd.read_csv(CACHE_FILE)
else:
    cache_df = pd.DataFrame(columns=["full_address", "lat", "lon"])

geolocator = Nominatim(user_agent="tyomatka_skanneri_demo")

def geocode_with_cache(address):
    row = cache_df.loc[cache_df["full_address"] == address]
    if not row.empty:
        return float(row.iloc[0]["lat"]), float(row.iloc[0]["lon"])
    try:
        loc = geolocator.geocode(address, timeout=10)
        if loc:
            lat, lon = loc.latitude, loc.longitude
        else:
            lat, lon = np.nan, np.nan
    except Exception:
        lat, lon = np.nan, np.nan

    cache_df.loc[len(cache_df)] = [address, lat, lon]
    cache_df.to_csv(CACHE_FILE, index=False)
    time.sleep(1.1)  # Nominatimin rajoituksia varten
    return lat, lon

print("\nGeokoodataan Mäntsälän osoitteet (vain ne, joilla katu löytyy)…")
coords = []
for addr in tqdm(df_with_street["full_address"], desc="Geocoding"):
    coords.append(geocode_with_cache(addr))

df_with_street[["lat", "lon"]] = pd.DataFrame(coords, index=df_with_street.index)
df_geocoded = df_with_street.dropna(subset=["lat", "lon"]).copy()
print(f"Geokoodattuja rivejä Mäntsälässä: {len(df_geocoded)}")
print(df_geocoded[["company_name", "full_address", "lat", "lon"]].head())

# --- 11. LASKETAAN matka-ajat Digitransit-rajapinnalta ---
HSL_QUERY = """
query($from: InputCoordinates!, $to: InputCoordinates!) {
  plan(
    from: $from,
    to: $to,
    modes: ["BUS","TRAIN","SUBWAY","TRAM","RAIL","WALK"],
    numItineraries: 1
  ) {
    itineraries { duration }
  }
}"""

def get_travel_time(home, target):
    payload = {
        "query": HSL_QUERY,
        "variables": {
            "from": home,
            "to":   {"lat": target[0], "lon": target[1]}
        }
    }
    try:
        r = requests.post(GRAPHQL_URL, json=payload,
                          headers={"User-Agent": USER_AGENT}, timeout=15)
        r.raise_for_status()
        data = r.json()
        its = data.get("data", {}).get("plan", {}).get("itineraries", [])
        if its:
            sec = its[0]["duration"]
            return round(sec / 60)
        return None
    except Exception:
        return None

print("\nLasketaan matka-ajat joukkoliikenteellä Mäntsälän asemalta…")
travel_times = []
for _, row in tqdm(df_geocoded.iterrows(), total=df_geocoded.shape[0], desc="Calculating travel time"):
    tt = get_travel_time(HOME_COORDS, (row["lat"], row["lon"]))
    travel_times.append(tt)
    time.sleep(0.3)

df_geocoded["travel_min"] = travel_times
df_with_time = df_geocoded.dropna(subset=["travel_min"]).copy()
df_with_time["travel_min"] = df_with_time["travel_min"].astype(int)

# --- 12. SUODATA alle 60 minuutin matka-ajan perusteella ---
MAX_MIN = 60
df_nearby = df_with_time[df_with_time["travel_min"] <= MAX_MIN].copy()
print(f"\n{len(df_nearby)} yritystä alle {MAX_MIN} minuutin matkan päässä:")
print(df_nearby[["company_name", "full_address", "travel_min"]].head())

# --- 13. TALLENNA lopputulos Exceliin ---
OUTPUT_FILE = "mansala_nearby.xlsx"
columns_to_save = ["company_name", "full_address", "lat", "lon", "travel_min", "tol_code"]
df_nearby.to_excel(OUTPUT_FILE, columns=columns_to_save, index=False)
print(f"\nTulokset tallennettu tiedostoon {OUTPUT_FILE}")


Haetaan yritykset, joilla postinumero = 04600…
Yrityksiä, joilla osoiterivi postinumero 04600: 100 riviä
Suodatuksen jälkeen 2 riviä (TOL‐prefix + postinumero 04600).

Suodatetut rivit osoitetiedoilla (enintään 5 riviä):
                                         company_name        street postcode  \
43  Mäntsälän K-tavaratalo Ostopörssi, Haarala R &...  KONKELOPOLKU    04600   
94                                Kaarlo Kuittinen Ky     KESKUSTIE    04600   

   postOffice  
43   MÄNTSÄLÄ  
94   MÄNTSÄLÄ  

Jäljelle katuosoitteiden perusteella: 2 riviä
                                         company_name        street postcode  \
43  Mäntsälän K-tavaratalo Ostopörssi, Haarala R &...  KONKELOPOLKU    04600   
94                                Kaarlo Kuittinen Ky     KESKUSTIE    04600   

   postOffice  
43   MÄNTSÄLÄ  
94   MÄNTSÄLÄ  

Geokoodataan Mäntsälän osoitteet (vain ne, joilla katu löytyy)…


Geocoding:   0%|          | 0/2 [00:00<?, ?it/s]

Geokoodattuja rivejä Mäntsälässä: 1
                                         company_name  \
43  Mäntsälän K-tavaratalo Ostopörssi, Haarala R &...   

                             full_address        lat        lon  
43  KONKELOPOLKU 04600 MÄNTSÄLÄ, Mäntsälä  60.641317  25.335062  

Lasketaan matka-ajat joukkoliikenteellä Mäntsälän asemalta…


Calculating travel time:   0%|          | 0/1 [00:00<?, ?it/s]


0 yritystä alle 60 minuutin matkan päässä:
Empty DataFrame
Columns: [company_name, full_address, travel_min]
Index: []

Tulokset tallennettu tiedostoon mansala_nearby.xlsx


In [ ]:
# --- 0. Asenna tarvittavat paketit (ajettava erikseen, jos ei vielä asennettuna) ---
# !pip install pandas requests geopy tqdm openpyxl

# --- 1. Tuodaan tarvittavat kirjastot ---
import requests
import pandas as pd
import numpy as np
import time
from tqdm.auto import tqdm
from geopy.geocoders import Nominatim
import os

# --- 2. MUUTTUJAT JA ASEMATIEDOT ---
BASE_URL     = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL     = f"{BASE_URL}/companies"
HEADERS      = {"Accept": "application/json"}

# Hakee kaikki yritykset, joilla osoiterivillä postinumero = 04600 (Mäntsälän alue)
MANTSALA_POSTCODE = "04600"
MAX_RESULTS       = 100  # 100 riittää, sillä Mäntsälässä osoiterivejä alle 100

# Digitransit‐reitityksen GraphQL‐endpoint
GRAPHQL_URL  = "https://api.digitransit.fi/routing/v1/routers/hsl/index/graphql"
USER_AGENT   = "tyomatka_skanneri_demo/1.0 (email@esimerkki.fi)"

# Juna‐asemat ja niiden koordinaatit (lat, lon)
# Voit halutessasi tarkentaa tarkat osoitekoordinaatit geokoodauksella tai muilla lähteillä
STATIONS = {
    "Mäntsälä": {"lat": 60.6320, "lon": 25.3187},
    "Kerava":   {"lat": 60.4027, "lon": 25.1028},
    "Lahti":    {"lat": 60.9827, "lon": 25.6570},
    "Helsinki": {"lat": 60.1719, "lon": 24.9410}
}

# --- 3. Hae kaikki yritykset postinumero 04600 (Mäntsälä) ---
print(f"Haetaan kaikki yritykset, joilla postinumero = {MANTSALA_POSTCODE}…")
params = {
    "postCode":    MANTSALA_POSTCODE,
    "maxResults":  MAX_RESULTS,
    "resultsFrom": 0
}
r = requests.get(COMP_URL, params=params, headers=HEADERS, timeout=15)
r.raise_for_status()
data = r.json()
companies = data.get("companies", [])
print(f"Yrityksiä, joilla osoiterivi 04600: {len(companies)} riviä")

# --- 4. Muunna DataFrameksi ja pura yritysnimi sekä osoitetiedot ---
df = pd.json_normalize(companies)

def get_company_name(names_list):
    """Ota yrityksen nimi names‐listasta (ensimmäinen elementti) tai None."""
    if isinstance(names_list, list) and names_list:
        return names_list[0].get("name", None)
    return None

df["company_name"] = df["names"].apply(get_company_name)

def extract_address_fields(addr_list):
    """
    Purkaa osoitelistan ensimmäisen dict‐olion ja palauttaa:
      'street'    : katuosoite
      'postcode'  : postinumero
      'postOffice': postitoimipaikka (kaupunki)
    Jos ei löydy, palauttaa None‐arvot.
    """
    if not isinstance(addr_list, list) or not addr_list:
        return {"street": None, "postcode": None, "postOffice": None}
    for addr in addr_list:
        if not isinstance(addr, dict):
            continue
        street = addr.get("street", None)
        postcode = addr.get("postCode", None)
        post_offices = addr.get("postOffices", [])
        post_office = None
        if isinstance(post_offices, list) and post_offices:
            post_office = post_offices[0].get("city", None)
        # Jos katu, postinumero ja postOffice löytyvät, palautetaan ne
        if street and postcode and post_office:
            return {"street": street, "postcode": postcode, "postOffice": post_office}
    # Jos ei löydy täydellistä riviä, ota ensimmäinen elementti
    first = addr_list[0]
    if isinstance(first, dict):
        street = first.get("street", None)
        postcode = first.get("postCode", None)
        post_offices = first.get("postOffices", [])
        post_office = None
        if isinstance(post_offices, list) and post_offices:
            post_office = post_offices[0].get("city", None)
        return {"street": street, "postcode": postcode, "postOffice": post_office}
    return {"street": None, "postcode": None, "postOffice": None}

# Luo sarakkeet
df["street"]     = df["addresses"].apply(lambda L: extract_address_fields(L)["street"])
df["postcode"]   = df["addresses"].apply(lambda L: extract_address_fields(L)["postcode"])
df["postOffice"] = df["addresses"].apply(lambda L: extract_address_fields(L)["postOffice"])

# Korvaa NaN tyhjillä merkkijonoilla
for col in ["street", "postcode", "postOffice"]:
    df[col] = df[col].fillna("")

print("\nEsimerkki suodatetuista riveistä osoitetiedoilla (max 5 riviä):")
print(df[["company_name", "street", "postcode", "postOffice"]].head(5))

# --- 5. Jätä vain rivit, joilla on katuosoite --}}
df_with_street = df[df["street"] != ""].copy()
print(f"\nJäljelle katuosoitteiden perusteella: {len(df_with_street)} riviä")
print(df_with_street[["company_name", "street", "postcode", "postOffice"]])

# --- 6. Luo 'full_address' geokoodaukselle ---
df_with_street["full_address"] = (
    df_with_street["street"] + " " +
    df_with_street["postcode"].astype(str) + " " +
    df_with_street["postOffice"] + ", Mäntsälä"
)

# --- 7. GEOKOODAA välimuistin avulla (Nominatim) ---
CACHE_FILE = "geocache.csv"
if os.path.exists(CACHE_FILE):
    cache_df = pd.read_csv(CACHE_FILE)
else:
    cache_df = pd.DataFrame(columns=["full_address", "lat", "lon"])

geolocator = Nominatim(user_agent="tyomatka_skanneri_demo")

def geocode_with_cache(address):
    """Geokoodaa osoitteen (lat, lon) tai hae välimuistista."""
    row = cache_df.loc[cache_df["full_address"] == address]
    if not row.empty:
        return float(row.iloc[0]["lat"]), float(row.iloc[0]["lon"])
    try:
        loc = geolocator.geocode(address, timeout=10)
        if loc:
            lat, lon = loc.latitude, loc.longitude
        else:
            lat, lon = np.nan, np.nan
    except Exception:
        lat, lon = np.nan, np.nan

    cache_df.loc[len(cache_df)] = [address, lat, lon]
    cache_df.to_csv(CACHE_FILE, index=False)
    time.sleep(1.1)  # Nominatim‐rajoitus ~1 pyyntö/s
    return lat, lon

print("\nGeokoodataan Mäntsälän osoitteet (vain ne, joilla katu löytyy)…")
coords = []
for addr in tqdm(df_with_street["full_address"], desc="Geocoding"):
    coords.append(geocode_with_cache(addr))

df_with_street[["lat", "lon"]] = pd.DataFrame(coords, index=df_with_street.index)
df_geocoded = df_with_street.dropna(subset=["lat", "lon"]).copy()
print(f"Geokoodattuja rivejä Mäntsälässä: {len(df_geocoded)}")
print(df_geocoded[["company_name", "full_address", "lat", "lon"]].head())

# --- 8. LASKETAAN matka‐ajat eri asemille ja valitaan lyhin --}}
HSL_QUERY = """
query($from: InputCoordinates!, $to: InputCoordinates!) {
  plan(
    from: $from,
    to: $to,
    modes: ["BUS","TRAIN","SUBWAY","TRAM","RAIL","WALK"],
    numItineraries: 1
  ) {
    itineraries { duration }
  }
}"""

def get_travel_time_to_any_station(home_coords, target_lat, target_lon):
    """
    Laskee matka‐ajan target_pisteestä jokaiseen STATIONS‐listalla
    olevaan asemaan ja palauttaa pienimmän ajan minuutteina.
    Jos yhtään reittiä ei löydy, palautetaan None.
    """
    times = []
    for station_name, station_coords in STATIONS.items():
        payload = {
            "query": HSL_QUERY,
            "variables": {
                "from": home_coords,
                "to":   {"lat": target_lat, "lon": target_lon}
            }
        }
        # Muutamme here "to" => ensin user antaa kohteen ja se haetaan
        # Jotta testi toimisi jokaiselle asemalle, vaihdamme vaihdekohdan:
        payload["variables"]["from"] = {"lat": target_lat, "lon": target_lon}
        payload["variables"]["to"] = station_coords

        try:
            r = requests.post(GRAPHQL_URL, json=payload,
                              headers={"User-Agent": USER_AGENT}, timeout=15)
            r.raise_for_status()
            data = r.json()
            its = data.get("data", {}).get("plan", {}).get("itineraries", [])
            if its:
                sec = its[0]["duration"]
                times.append(round(sec / 60))
            else:
                times.append(None)
        except Exception:
            times.append(None)

        time.sleep(0.3)  # Pieni viive kutsujen välissä

    # Suodatetaan pois None‐arvot
    valid = [t for t in times if t is not None]
    return min(valid) if valid else None

print("\nLasketaan matka‐ajat lähimmälle asemalle…")
travel_times = []
for _, row in tqdm(df_geocoded.iterrows(), total=df_geocoded.shape[0], desc="Calculating travel time"):
    tt = get_travel_time_to_any_station(
        HOME_COORDS, row["lat"], row["lon"]
    )
    travel_times.append(tt)

df_geocoded["travel_min"] = travel_times
df_with_time = df_geocoded.dropna(subset=["travel_min"]).copy()
df_with_time["travel_min"] = df_with_time["travel_min"].astype(int)

# --- 9. SUODATA ne, joiden matka‐aika ≤ 60 min --}}
MAX_MIN = 60
df_nearby = df_with_time[df_with_time["travel_min"] <= MAX_MIN].copy()
print(f"\n{len(df_nearby)} yritystä alle {MAX_MIN} min matkan päässä lähimmältä asemalta:")
print(df_nearby[["company_name", "full_address", "travel_min"]].head())

# --- 10. TALLENNA lopputulos Exceliin --}}
OUTPUT_FILE = "mansala_nearby.xlsx"
columns_to_save = ["company_name", "full_address", "lat", "lon", "travel_min"]
df_nearby.to_excel(OUTPUT_FILE, columns=columns_to_save, index=False)
print(f"\nTulokset tallennettu tiedostoon {OUTPUT_FILE}")


Haetaan kaikki yritykset, joilla postinumero = 04600…
Yrityksiä, joilla osoiterivi 04600: 100 riviä

Esimerkki suodatetuista riveistä osoitetiedoilla (max 5 riviä):
                          company_name          street postcode postOffice
0  Asunto Oy Askolan Riihipellontie 16  Riihipellontie    07500     ASKOLA
1             Asunto Oy Ojasillantie 4    OJASILLANTIE    07230     ASKOLA
2     Asunto-osakeyhtiö Ojasillantie 2    Ojasillantie    07230     ASKOLA
3  Asunto-osakeyhtiö Riihipellontie 12  Riihipellontie    07500     ASKOLA
4               Asunto Oy Haukivalkama      Keskuskatu    04600   MÄNTSÄLÄ

Jäljelle katuosoitteiden perusteella: 100 riviä
                           company_name           street postcode   postOffice
0   Asunto Oy Askolan Riihipellontie 16   Riihipellontie    07500       ASKOLA
1              Asunto Oy Ojasillantie 4     OJASILLANTIE    07230       ASKOLA
2      Asunto-osakeyhtiö Ojasillantie 2     Ojasillantie    07230       ASKOLA
3   Asunto-osakeyhti

Geocoding:   0%|          | 0/100 [00:00<?, ?it/s]

Geokoodattuja rivejä Mäntsälässä: 72
                   company_name                         full_address  \
4        Asunto Oy Haukivalkama  Keskuskatu 04600 MÄNTSÄLÄ, Mäntsälä   
7      Asunto Oy Isonnevantie 4  Keskuskatu 04600 MÄNTSÄLÄ, Mäntsälä   
9     Asunto Oy Haavikkopolku 1  Keskuskatu 04600 MÄNTSÄLÄ, Mäntsälä   
10  As. Oy Keravan Nissilänpiha  Keskuskatu 04600 MÄNTSÄLÄ, Mäntsälä   
11       Kiinteistö Oy Huikkola    OSUUSTIE 04600 MÄNTSÄLÄ, Mäntsälä   

          lat        lon  
4   60.637778  25.327635  
7   60.637778  25.327635  
9   60.637778  25.327635  
10  60.637778  25.327635  
11  60.635503  25.315708  

Lasketaan matka‐ajat lähimmälle asemalle…


Calculating travel time:   0%|          | 0/72 [00:00<?, ?it/s]


0 yritystä alle 60 min matkan päässä lähimmältä asemalta:
Empty DataFrame
Columns: [company_name, full_address, travel_min]
Index: []

Tulokset tallennettu tiedostoon mansala_nearby.xlsx


In [ ]:
# --- 0. Asenna tarvittavat paketit (ajettava erikseen, jos ei vielä asennettuna) ---
# !pip install pandas requests geopy tqdm openpyxl

# --- 1. Tuodaan tarvittavat kirjastot ---
import requests
import pandas as pd
import numpy as np
import time
from tqdm.auto import tqdm
from geopy.geocoders import Nominatim
import os

# --- 2. MUUTTUJAT JA ASEMATIEDOT ---
BASE_URL     = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL     = f"{BASE_URL}/companies"
HEADERS      = {"Accept": "application/json"}

# Hakee kaikki yritykset, joilla osoiterivillä postinumero = 04600 (Mäntsälän alue)
MANTSALA_POSTCODE = "04600"
MAX_RESULTS       = 100  # 100 riittää, sillä Mäntsälässä osoiterivejä alle 100

# Digitransit‐reitityksen GraphQL‐endpoint
GRAPHQL_URL  = "https://api.digitransit.fi/routing/v1/routers/hsl/index/graphql"
USER_AGENT   = "tyomatka_skanneri_demo/1.0 (email@esimerkki.fi)"

# Juna‐asemat ja niiden koordinaatit (lat, lon)
# Voit halutessasi tarkentaa tarkat osoitekoordinaatit geokoodauksella tai muilla lähteillä
STATIONS = {
    "Mäntsälä": {"lat": 60.6320, "lon": 25.3187},
    "Kerava":   {"lat": 60.4027, "lon": 25.1028},
    "Lahti":    {"lat": 60.9827, "lon": 25.6570},
    "Helsinki": {"lat": 60.1719, "lon": 24.9410}
}

# --- 3. Hae kaikki yritykset postinumero 04600 (Mäntsälä) ---
print(f"Haetaan kaikki yritykset, joilla postinumero = {MANTSALA_POSTCODE}…")
params = {
    "postCode":    MANTSALA_POSTCODE,
    "maxResults":  MAX_RESULTS,
    "resultsFrom": 0
}
r = requests.get(COMP_URL, params=params, headers=HEADERS, timeout=15)
r.raise_for_status()
data = r.json()
companies = data.get("companies", [])
print(f"Yrityksiä, joilla osoiterivi 04600: {len(companies)} riviä")

# --- 4. Muunna DataFrameksi ja pura yritysnimi sekä osoitetiedot ---
df = pd.json_normalize(companies)

def get_company_name(names_list):
    """Ota yrityksen nimi names‐listasta (ensimmäinen elementti) tai None."""
    if isinstance(names_list, list) and names_list:
        return names_list[0].get("name", None)
    return None

df["company_name"] = df["names"].apply(get_company_name)

def extract_address_fields(addr_list):
    """
    Purkaa osoitelistan ensimmäisen dict‐olion ja palauttaa:
      'street'    : katuosoite
      'postcode'  : postinumero
      'postOffice': postitoimipaikka (kaupunki)
    Jos ei löydy, palauttaa None‐arvot.
    """
    if not isinstance(addr_list, list) or not addr_list:
        return {"street": None, "postcode": None, "postOffice": None}
    for addr in addr_list:
        if not isinstance(addr, dict):
            continue
        street = addr.get("street", None)
        postcode = addr.get("postCode", None)
        post_offices = addr.get("postOffices", [])
        post_office = None
        if isinstance(post_offices, list) and post_offices:
            post_office = post_offices[0].get("city", None)
        # Jos katu, postinumero ja postOffice löytyvät, palautetaan ne
        if street and postcode and post_office:
            return {"street": street, "postcode": postcode, "postOffice": post_office}
    # Jos ei löydy täydellistä riviä, ota ensimmäinen elementti
    first = addr_list[0]
    if isinstance(first, dict):
        street = first.get("street", None)
        postcode = first.get("postCode", None)
        post_offices = first.get("postOffices", [])
        post_office = None
        if isinstance(post_offices, list) and post_offices:
            post_office = post_offices[0].get("city", None)
        return {"street": street, "postcode": postcode, "postOffice": post_office}
    return {"street": None, "postcode": None, "postOffice": None}

# Luo sarakkeet
df["street"]     = df["addresses"].apply(lambda L: extract_address_fields(L)["street"])
df["postcode"]   = df["addresses"].apply(lambda L: extract_address_fields(L)["postcode"])
df["postOffice"] = df["addresses"].apply(lambda L: extract_address_fields(L)["postOffice"])

# Korvaa NaN tyhjillä merkkijonoilla
for col in ["street", "postcode", "postOffice"]:
    df[col] = df[col].fillna("")

print("\nEsimerkki suodatetuista riveistä osoitetiedoilla (max 5 riviä):")
print(df[["company_name", "street", "postcode", "postOffice"]].head(5))

# --- 5. Jätä vain rivit, joilla on katuosoite --}}
df_with_street = df[df["street"] != ""].copy()
print(f"\nJäljelle katuosoitteiden perusteella: {len(df_with_street)} riviä")
print(df_with_street[["company_name", "street", "postcode", "postOffice"]])

# --- 6. Luo 'full_address' geokoodaukselle ---
df_with_street["full_address"] = (
    df_with_street["street"] + " " +
    df_with_street["postcode"].astype(str) + " " +
    df_with_street["postOffice"] + ", Mäntsälä"
)

# --- 7. GEOKOODAA välimuistin avulla (Nominatim) ---
CACHE_FILE = "geocache.csv"
if os.path.exists(CACHE_FILE):
    cache_df = pd.read_csv(CACHE_FILE)
else:
    cache_df = pd.DataFrame(columns=["full_address", "lat", "lon"])

geolocator = Nominatim(user_agent="tyomatka_skanneri_demo")

def geocode_with_cache(address):
    """Geokoodaa osoitteen (lat, lon) tai hae välimuistista."""
    row = cache_df.loc[cache_df["full_address"] == address]
    if not row.empty:
        return float(row.iloc[0]["lat"]), float(row.iloc[0]["lon"])
    try:
        loc = geolocator.geocode(address, timeout=10)
        if loc:
            lat, lon = loc.latitude, loc.longitude
        else:
            lat, lon = np.nan, np.nan
    except Exception:
        lat, lon = np.nan, np.nan

    cache_df.loc[len(cache_df)] = [address, lat, lon]
    cache_df.to_csv(CACHE_FILE, index=False)
    time.sleep(1.1)  # Nominatim‐rajoitus ~1 pyyntö/s
    return lat, lon

print("\nGeokoodataan Mäntsälän osoitteet (vain ne, joilla katu löytyy)…")
coords = []
for addr in tqdm(df_with_street["full_address"], desc="Geocoding"):
    coords.append(geocode_with_cache(addr))

df_with_street[["lat", "lon"]] = pd.DataFrame(coords, index=df_with_street.index)
df_geocoded = df_with_street.dropna(subset=["lat", "lon"]).copy()
print(f"Geokoodattuja rivejä Mäntsälässä: {len(df_geocoded)}")
print(df_geocoded[["company_name", "full_address", "lat", "lon"]].head())

# --- 8. LASKETAAN matka‐ajat eri asemille ja valitaan lyhin --}}
HSL_QUERY = """
query($from: InputCoordinates!, $to: InputCoordinates!) {
  plan(
    from: $from,
    to: $to,
    modes: ["BUS","TRAIN","SUBWAY","TRAM","RAIL","WALK"],
    numItineraries: 1
  ) {
    itineraries { duration }
  }
}"""

def get_travel_time_to_any_station(home_coords, target_lat, target_lon):
    times = []
    for station_name, station_coords in STATIONS.items():
        payload = {
            "query": HSL_QUERY,
            "variables": {
                "from": {"lat": target_lat, "lon": target_lon},
                "to":   station_coords
            }
        }
        try:
            r = requests.post(GRAPHQL_URL, json=payload,
                              headers={"User-Agent": USER_AGENT}, timeout=15)
            r.raise_for_status()
            data = r.json()
            its = data.get("data", {}).get("plan", {}).get("itineraries", [])
            if its:
                sec = its[0]["duration"]
                times.append(round(sec / 60))
            else:
                times.append(None)
        except Exception:
            times.append(None)

        time.sleep(0.3)

    valid = [t for t in times if t is not None]
    return min(valid) if valid else None


print("\nLasketaan matka‐ajat lähimmälle asemalle…")
travel_times = []
for _, row in tqdm(df_geocoded.iterrows(), total=df_geocoded.shape[0], desc="Calculating travel time"):
    tt = get_travel_time_to_any_station(
        HOME_COORDS, row["lat"], row["lon"]
    )
    travel_times.append(tt)

df_geocoded["travel_min"] = travel_times
df_with_time = df_geocoded.dropna(subset=["travel_min"]).copy()
df_with_time["travel_min"] = df_with_time["travel_min"].astype(int)

# Tulosta kaikki yritykset ja niiden laskettu matka‐aika lähimmälle asemalle
print("Yritykset ja niiden matka‐ajat (min) lähimpään asemaan:")
print(df_with_time[["company_name", "full_address", "travel_min"]])

# Jos haluat myös tallentaa tämän listan erikseen:
df_with_time.to_excel("mansala_all_travel_times.xlsx",
                     columns=["company_name", "full_address", "lat", "lon", "travel_min"],
                     index=False)
print("Kaikki matka‐ajat tallennettu mansala_all_travel_times.xlsx")

# Sitten voit vasta suodattaa alle 60 min:
MAX_MIN = 60
df_nearby = df_with_time[df_with_time["travel_min"] <= MAX_MIN].copy()
print(f"\n{len(df_nearby)} yritystä alle {MAX_MIN} min matkan päässä:")
print(df_nearby[["company_name", "full_address", "travel_min"]])


Haetaan kaikki yritykset, joilla postinumero = 04600…
Yrityksiä, joilla osoiterivi 04600: 100 riviä

Esimerkki suodatetuista riveistä osoitetiedoilla (max 5 riviä):
                          company_name          street postcode postOffice
0  Asunto Oy Askolan Riihipellontie 16  Riihipellontie    07500     ASKOLA
1             Asunto Oy Ojasillantie 4    OJASILLANTIE    07230     ASKOLA
2     Asunto-osakeyhtiö Ojasillantie 2    Ojasillantie    07230     ASKOLA
3  Asunto-osakeyhtiö Riihipellontie 12  Riihipellontie    07500     ASKOLA
4               Asunto Oy Haukivalkama      Keskuskatu    04600   MÄNTSÄLÄ

Jäljelle katuosoitteiden perusteella: 100 riviä
                           company_name           street postcode   postOffice
0   Asunto Oy Askolan Riihipellontie 16   Riihipellontie    07500       ASKOLA
1              Asunto Oy Ojasillantie 4     OJASILLANTIE    07230       ASKOLA
2      Asunto-osakeyhtiö Ojasillantie 2     Ojasillantie    07230       ASKOLA
3   Asunto-osakeyhti

Geocoding:   0%|          | 0/100 [00:00<?, ?it/s]

Geokoodattuja rivejä Mäntsälässä: 72
                   company_name                         full_address  \
4        Asunto Oy Haukivalkama  Keskuskatu 04600 MÄNTSÄLÄ, Mäntsälä   
7      Asunto Oy Isonnevantie 4  Keskuskatu 04600 MÄNTSÄLÄ, Mäntsälä   
9     Asunto Oy Haavikkopolku 1  Keskuskatu 04600 MÄNTSÄLÄ, Mäntsälä   
10  As. Oy Keravan Nissilänpiha  Keskuskatu 04600 MÄNTSÄLÄ, Mäntsälä   
11       Kiinteistö Oy Huikkola    OSUUSTIE 04600 MÄNTSÄLÄ, Mäntsälä   

          lat        lon  
4   60.637778  25.327635  
7   60.637778  25.327635  
9   60.637778  25.327635  
10  60.637778  25.327635  
11  60.635503  25.315708  

Lasketaan matka‐ajat lähimmälle asemalle…


Calculating travel time:   0%|          | 0/72 [00:00<?, ?it/s]

Yritykset ja niiden matka‐ajat (min) lähimpään asemaan:
Empty DataFrame
Columns: [company_name, full_address, travel_min]
Index: []
Kaikki matka‐ajat tallennettu mansala_all_travel_times.xlsx

0 yritystä alle 60 min matkan päässä:
Empty DataFrame
Columns: [company_name, full_address, travel_min]
Index: []


In [ ]:
# --- 0. Asenna tarvittavat paketit (ajettava erikseen, jos eivät ole jo asennettuna) ---
# !pip install pandas requests geopy tqdm openpyxl

# --- 1. Tuodaan tarvittavat kirjastot ---
import requests
import pandas as pd
import numpy as np
import time
from tqdm.auto import tqdm
from geopy.geocoders import Nominatim
import math
import os

# --- 2. ASETUKSET JA MUUTTUJAT ---
BASE_URL     = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL     = f"{BASE_URL}/companies"
HEADERS      = {"Accept": "application/json"}

# Haetaan kaikki yritykset, joilla osoiterivillä postinumero = 04600 (Mäntsälän alue)
MANTSALA_POSTCODE = "04600"
MAX_RESULTS       = 100  # riittää, sillä osoiterivejä alle 100

# Juna-asemat ja niiden koordinaatit (lat, lon)
STATIONS = {
    "Mäntsälä": {"lat": 60.6320, "lon": 25.3187},
    "Kerava":   {"lat": 60.4027, "lon": 25.1028},
    "Lahti":    {"lat": 60.9827, "lon": 25.6570},
    "Helsinki": {"lat": 60.1719, "lon": 24.9410}
}

# Kävelynopeus km/h → 5 km/h (noin 12 min per km)
WALKING_SPEED_KMH = 5

# --- 3. Hae kaikki yritykset, joiden osoiterivillä postinumero = 04600 ---
print(f"Haetaan yritykset, joilla postinumero = {MANTSALA_POSTCODE}…")
params = {
    "postCode":    MANTSALA_POSTCODE,
    "maxResults":  MAX_RESULTS,
    "resultsFrom": 0
}
r = requests.get(COMP_URL, params=params, headers=HEADERS, timeout=15)
r.raise_for_status()
data = r.json()
companies = data.get("companies", [])
print(f"Yrityksiä, joilla osoiterivi {MANTSALA_POSTCODE}: {len(companies)} riviä")

# --- 4. Muunna DataFrameksi ja pura nimi + osoitetiedot ---
df = pd.json_normalize(companies)

def get_company_name(names_list):
    """
    Palauttaa yrityksen nimen names-listasta (ensimmäinen elementti),
    tai None jos ei löydy.
    """
    if isinstance(names_list, list) and names_list:
        return names_list[0].get("name", None)
    return None

df["company_name"] = df["names"].apply(get_company_name)

def extract_address_fields(addr_list):
    """
    Purkaa osoitelistan ensimmäisen dict-olion ja palauttaa:
      - 'street'    : katuosoite
      - 'postcode'  : postinumero
      - 'postOffice': postitoimipaikka (kaupunki)
    Jos ei löydy, palauttaa None-arvot.
    """
    if not isinstance(addr_list, list) or not addr_list:
        return {"street": None, "postcode": None, "postOffice": None}
    for addr in addr_list:
        if not isinstance(addr, dict):
            continue
        street = addr.get("street", None)
        postcode = addr.get("postCode", None)
        post_offices = addr.get("postOffices", [])
        post_office = None
        if isinstance(post_offices, list) and post_offices:
            post_office = post_offices[0].get("city", None)
        # Jos katu, postinumero ja postOffice löytyvät, palautetaan ne
        if street and postcode and post_office:
            return {"street": street, "postcode": postcode, "postOffice": post_office}
    # Jos ei löytynyt täydellistä, ota ensimmäinen
    first = addr_list[0]
    if isinstance(first, dict):
        street = first.get("street", None)
        postcode = first.get("postCode", None)
        post_offices = first.get("postOffices", [])
        post_office = None
        if isinstance(post_offices, list) and post_offices:
            post_office = post_offices[0].get("city", None)
        return {"street": street, "postcode": postcode, "postOffice": post_office}
    return {"street": None, "postcode": None, "postOffice": None}

# Luo uudet sarakkeet
df["street"]     = df["addresses"].apply(lambda L: extract_address_fields(L)["street"])
df["postcode"]   = df["addresses"].apply(lambda L: extract_address_fields(L)["postcode"])
df["postOffice"] = df["addresses"].apply(lambda L: extract_address_fields(L)["postOffice"])

# Korvaa NaN tyhjillä merkkijonoilla
for col in ["street", "postcode", "postOffice"]:
    df[col] = df[col].fillna("")

print("\nEsimerkki riveistä osoitetiedoilla (max 5 riviä):")
print(df[["company_name", "street", "postcode", "postOffice", "mainBusinessLine.type"]].head(5))

# --- 5. POISTA asunto- ja kiinteistöyhtiöt nimestä ---
# Suodatetaan pois, jos nimi alkaa yhdellä seuraavista (case-insensitive):
#   Asunto Oy, Asunto-osakeyhtiö, Asunto-Oy, As. Oy, As Oy, As.Oy
#   Kiinteistö Oy, Kiinteistöosakeyhtiö
mask_housing = df["company_name"].str.match(
    r"^(Asunto[\- ]?Oy|Asunto[\- ]?osakeyhtiö|As\.? ?Oy|As\.Oy|AsOy|Kiinteistö[\- ]?Oy|Kiinteistöosakeyhtiö)",
    case=False,
    na=False
)
df = df.loc[~mask_housing].copy()
print(f"\nAsunto- ja Kiinteistö-yhtiöt poistettu, jäljellä: {len(df)} riviä")

# --- 6. Jätä vain yritykset, joilla on katuosoite (tarvitaan geokoodaukseen) ---
df_with_street = df[df["street"] != ""].copy()
print(f"\nJäljelle katuosoitteiden mukaan: {len(df_with_street)} riviä")
print(df_with_street[["company_name", "street", "postcode", "postOffice"]])

# --- 7. Luo 'full_address' geokoodaukselle ---
df_with_street["full_address"] = (
    df_with_street["street"] + " " +
    df_with_street["postcode"].astype(str) + " " +
    df_with_street["postOffice"] + ", Mäntsälä"
)

# --- 8. GEOKOODAA välimuistin avulla (Nominatim) ---
CACHE_FILE = "geocache.csv"
if os.path.exists(CACHE_FILE):
    cache_df = pd.read_csv(CACHE_FILE)
else:
    cache_df = pd.DataFrame(columns=["full_address", "lat", "lon"])

geolocator = Nominatim(user_agent="tyomatka_skanneri_demo")

def geocode_with_cache(address):
    """
    Geokoodaa osoitteen (lat, lon) tai hakee aikaisemman tuloksen cacheista.
    Tallentaa uudet osoitteet 'geocache.csv'.
    """
    row = cache_df.loc[cache_df["full_address"] == address]
    if not row.empty:
        return float(row.iloc[0]["lat"]), float(row.iloc[0]["lon"])
    try:
        loc = geolocator.geocode(address, timeout=10)
        if loc:
            lat, lon = loc.latitude, loc.longitude
        else:
            lat, lon = np.nan, np.nan
    except Exception:
        lat, lon = np.nan, np.nan

    cache_df.loc[len(cache_df)] = [address, lat, lon]
    cache_df.to_csv(CACHE_FILE, index=False)
    time.sleep(1.1)  # Nominatimin rajoitus ~1 pyyntö/s
    return lat, lon

print("\nGeokoodataan Mäntsälän osoitteet (vain ne, joilla katu löytyy)…")
coords = []
for addr in tqdm(df_with_street["full_address"], desc="Geocoding"):
    coords.append(geocode_with_cache(addr))

df_with_street[["lat", "lon"]] = pd.DataFrame(coords, index=df_with_street.index)
df_geocoded = df_with_street.dropna(subset=["lat", "lon"]).copy()
print(f"Geokoodattuja rivejä Mäntsälässä: {len(df_geocoded)}")
print(df_geocoded[["company_name", "full_address", "lat", "lon", "mainBusinessLine.type"]].head())

# --- 9. Haversine-etäisyys ja kävelyaika lähimpään asemalle ---
def haversine(lat1, lon1, lat2, lon2):
    """Laskee etäisyyden km kahden koordinaattipisteen välillä Haversine-kaavalla."""
    R = 6371.0  # Maan säde kilometreinä
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi  = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2.0)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2.0)**2
    return 2 * R * math.asin(math.sqrt(a))

print("\nLasketaan etäisyydet ja kävelyajat lähimpään asemaan…")
distances = []
for _, row in tqdm(df_geocoded.iterrows(), total=df_geocoded.shape[0], desc="Calculating distances"):
    lat_c, lon_c = row["lat"], row["lon"]
    # Laske etäisyys jokaiselle asemalle, valitse pienin
    min_dist_km = None
    for station_coords in STATIONS.values():
        km = haversine(lat_c, lon_c, station_coords["lat"], station_coords["lon"])
        if min_dist_km is None or km < min_dist_km:
            min_dist_km = km
    # Muunna kävelyajaksi minuuteiksi: (km / nopeus_km/h) * 60
    walk_min = round((min_dist_km / WALKING_SPEED_KMH) * 60) if min_dist_km is not None else None
    distances.append(walk_min)

df_geocoded["travel_min"] = distances

print("\nYritykset ja niiden kävelyaika lähimpään asemaan:")
print(df_geocoded[["company_name", "full_address", "travel_min", "mainBusinessLine.type"]])

# --- 10. SUODATA ne, joiden kävelyaika ≤ 60 minuuttia ---
MAX_MIN = 60
df_nearby = df_geocoded[df_geocoded["travel_min"] <= MAX_MIN].copy()
print(f"\n{len(df_nearby)} yritystä alle {MAX_MIN} min kävelymatkan päässä asemalta:")
print(df_nearby[["company_name", "full_address", "travel_min", "mainBusinessLine.type"]])

# --- 11. TALLENNA lopputulos Exceliin (kaikki sarakkeet mukana) ---
OUTPUT_FILE = "mansala_nearby_all_columns.xlsx"
df_nearby.to_excel(OUTPUT_FILE, index=False)
print(f"\nTulokset tallennettu tiedostoon {OUTPUT_FILE} (kaikki sarakkeet mukana)")


Haetaan yritykset, joilla postinumero = 04600…
Yrityksiä, joilla osoiterivi 04600: 100 riviä

Esimerkki riveistä osoitetiedoilla (max 5 riviä):
                          company_name          street postcode postOffice  \
0  Asunto Oy Askolan Riihipellontie 16  Riihipellontie    07500     ASKOLA   
1             Asunto Oy Ojasillantie 4    OJASILLANTIE    07230     ASKOLA   
2     Asunto-osakeyhtiö Ojasillantie 2    Ojasillantie    07230     ASKOLA   
3  Asunto-osakeyhtiö Riihipellontie 12  Riihipellontie    07500     ASKOLA   
4               Asunto Oy Haukivalkama      Keskuskatu    04600   MÄNTSÄLÄ   

  mainBusinessLine.type  
0                 68202  
1                 68202  
2                 68202  
3                 68202  
4                 68202  

Asunto- ja Kiinteistö-yhtiöt poistettu, jäljellä: 15 riviä

Jäljelle katuosoitteiden mukaan: 15 riviä
                                         company_name           street  \
6                                       Kellonosat Oy 

Geocoding:   0%|          | 0/15 [00:00<?, ?it/s]

Geokoodattuja rivejä Mäntsälässä: 13
                                         company_name  \
12                                    Nivos Verkot Oy   
13                         OP Koti Ylä-Uusimaa Oy LKV   
37                    Kommandiittiyhtiö Aimo Kiikkilä   
41                               Fennica-Huonekalu Oy   
43  Mäntsälän K-tavaratalo Ostopörssi, Haarala R &...   

                               full_address        lat        lon  \
12        SEPÄNTIE 04600 MÄNTSÄLÄ, Mäntsälä  60.639791  25.323942   
13      Keskuskatu 04600 MÄNTSÄLÄ, Mäntsälä  60.637778  25.327635   
37      Keskuskatu 04600 MÄNTSÄLÄ, Mäntsälä  60.637778  25.327635   
41  KENKÄPELLONTIE 04600 MÄNTSÄLÄ, Mäntsälä  60.647848  25.332477   
43    KONKELOPOLKU 04600 MÄNTSÄLÄ, Mäntsälä  60.641317  25.335062   

   mainBusinessLine.type  
12                 35120  
13                 68310  
37                 81100  
41                 68209  
43                 47111  

Lasketaan etäisyydet ja kävelyajat lähimpä

Calculating distances:   0%|          | 0/13 [00:00<?, ?it/s]


Yritykset ja niiden kävelyaika lähimpään asemaan:
                                         company_name  \
12                                    Nivos Verkot Oy   
13                         OP Koti Ylä-Uusimaa Oy LKV   
37                    Kommandiittiyhtiö Aimo Kiikkilä   
41                               Fennica-Huonekalu Oy   
43  Mäntsälän K-tavaratalo Ostopörssi, Haarala R &...   
52                    Kuljetusliike Teuvo Saarinen Oy   
53                                  Mäntsälän Saha Oy   
77           Mäntsälän Palvelutaksi Kommandiittiyhtiö   
80                 Mäntsälän Autopurkaamo Avoin yhtiö   
81                          Mairuen Kone ja Pultti Oy   
87                                       Lomasalmi Oy   
95                            J. Pajun Autotarvike Ky   
97                                    Marila Infra Oy   

                                full_address  travel_min mainBusinessLine.type  
12         SEPÄNTIE 04600 MÄNTSÄLÄ, Mäntsälä          11             

In [ ]:
# --- 0. Asenna tarvittavat paketit (ajettava erikseen, jos eivät vielä asennettuja) ---
# !pip install pandas requests geopy tqdm openpyxl

# --- 1. Tuodaan tarvittavat kirjastot ---
import requests
import pandas as pd
import numpy as np
import time
from tqdm.auto import tqdm
from geopy.geocoders import Nominatim
import math
import os

# --- 2. ASETUKSET JA MUUTTUJAT ---
BASE_URL     = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL     = f"{BASE_URL}/companies"
HEADERS      = {"Accept": "application/json"}

# Haetaan kaikki yritykset, joilla osoiterivillä postinumero = 04600 (Mäntsälän alue)
MANTSALA_POSTCODE = "04600"
MAX_RESULTS       = 100  # riittää, sillä Mäntsälässä alle 100 osoiteriviä

# Juna-asemat ja niiden koordinaatit (lat, lon)
STATIONS = {
    "Mäntsälä": {"lat": 60.6320, "lon": 25.3187},
    "Kerava":   {"lat": 60.4027, "lon": 25.1028},
    "Lahti":    {"lat": 60.9827, "lon": 25.6570},
    "Helsinki": {"lat": 60.1719, "lon": 24.9410}
}

# Kävelynopeus km/h → 5 km/h (noin 12 min/km)
WALKING_SPEED_KMH = 5

# --- 3. Hae yritykset postinumero 04600 ---
print(f"Haetaan yritykset, joilla postinumero = {MANTSALA_POSTCODE}…")
params = {
    "postCode":    MANTSALA_POSTCODE,
    "maxResults":  MAX_RESULTS,
    "resultsFrom": 0
}
r = requests.get(COMP_URL, params=params, headers=HEADERS, timeout=15)
r.raise_for_status()
data = r.json()
companies = data.get("companies", [])
print(f"Yrityksiä, joilla osoiterivi {MANTSALA_POSTCODE}: {len(companies)} riviä")

# --- 4. Muunna DataFrameksi ja pura nimi + osoitetiedot sekä muut kentät jäävät ennalleen ---
df = pd.json_normalize(companies)

# Pura yrityksen nimi
def get_company_name(names_list):
    if isinstance(names_list, list) and names_list:
        return names_list[0].get("name", None)
    return None

df["company_name"] = df["names"].apply(get_company_name)

# Pura osoitetiedot (street, postcode, postOffice)
def extract_address_fields(addr_list):
    if not isinstance(addr_list, list) or not addr_list:
        return {"street": None, "postcode": None, "postOffice": None}
    for addr in addr_list:
        if not isinstance(addr, dict):
            continue
        street = addr.get("street", None)
        postcode = addr.get("postCode", None)
        post_offices = addr.get("postOffices", [])
        post_office = None
        if isinstance(post_offices, list) and post_offices:
            post_office = post_offices[0].get("city", None)
        if street and postcode and post_office:
            return {"street": street, "postcode": postcode, "postOffice": post_office}
    first = addr_list[0]
    if isinstance(first, dict):
        street = first.get("street", None)
        postcode = first.get("postCode", None)
        post_offices = first.get("postOffices", [])
        post_office = None
        if isinstance(post_offices, list) and post_offices:
            post_office = post_offices[0].get("city", None)
        return {"street": street, "postcode": postcode, "postOffice": post_office}
    return {"street": None, "postcode": None, "postOffice": None}

df["street"]     = df["addresses"].apply(lambda L: extract_address_fields(L)["street"])
df["postcode"]   = df["addresses"].apply(lambda L: extract_address_fields(L)["postcode"])
df["postOffice"] = df["addresses"].apply(lambda L: extract_address_fields(L)["postOffice"])

for col in ["street", "postcode", "postOffice"]:
    df[col] = df[col].fillna("")

print("\nEsimerkki riveistä osoitetiedoilla (max 5 riviä):")
print(df[["company_name", "street", "postcode", "postOffice"]].head(5))

# --- 5. Poista asunto- ja kiinteistöyhtiöt nimestä (myös “As. Oy” tai “As Oy”) ---
mask_housing = df["company_name"].str.match(
    r"^(Asunto[\- ]?Oy|Asunto[\- ]?osakeyhtiö|As\.? ?Oy|As\.Oy|AsOy|Kiinteistö[\- ]?Oy|Kiinteistöosakeyhtiö)",
    case=False,
    na=False
)
df = df.loc[~mask_housing].copy()
print(f"\nAsunto- ja Kiinteistö-yhtiöt poistettu, jäljellä: {len(df)} riviä")

# --- 6. Jätä vain rivit, joilla on katuosoite ---
df_with_street = df[df["street"] != ""].copy()
print(f"\nJäljelle katuosoitteiden mukaan: {len(df_with_street)} riviä")

# --- 7. Luo 'full_address' geokoodaukselle ---
df_with_street["full_address"] = (
    df_with_street["street"] + " " +
    df_with_street["postcode"].astype(str) + " " +
    df_with_street["postOffice"] + ", Mäntsälä"
)

# --- 8. Geokoodaa välimuistin avulla (Nominatim) ---
CACHE_FILE = "geocache.csv"
if os.path.exists(CACHE_FILE):
    cache_df = pd.read_csv(CACHE_FILE)
else:
    cache_df = pd.DataFrame(columns=["full_address", "lat", "lon"])

geolocator = Nominatim(user_agent="tyomatka_skanneri_demo")

def geocode_with_cache(address):
    """Geokoodaa osoitteen tai palauttaa aikaisemman arvon cacheista."""
    row = cache_df.loc[cache_df["full_address"] == address]
    if not row.empty:
        return float(row.iloc[0]["lat"]), float(row.iloc[0]["lon"])
    try:
        loc = geolocator.geocode(address, timeout=10)
        if loc:
            lat, lon = loc.latitude, loc.longitude
        else:
            lat, lon = np.nan, np.nan
    except Exception:
        lat, lon = np.nan, np.nan

    cache_df.loc[len(cache_df)] = [address, lat, lon]
    cache_df.to_csv(CACHE_FILE, index=False)
    time.sleep(1.1)  # Nominatim‐rajoitus ~1 pyyntö/s
    return lat, lon

print("\nGeokoodataan Mäntsälän osoitteet (vain ne, joilla katu löytyy)…")
coords = []
for addr in tqdm(df_with_street["full_address"], desc="Geocoding"):
    coords.append(geocode_with_cache(addr))

df_with_street[["lat", "lon"]] = pd.DataFrame(coords, index=df_with_street.index)
df_geocoded = df_with_street.dropna(subset=["lat", "lon"]).copy()
print(f"Geokoodattuja rivejä Mäntsälässä: {len(df_geocoded)}")

# --- 9. Haversine‐etäisyys ja kävelyaika lähimpään asemalle ---
def haversine(lat1, lon1, lat2, lon2):
    """Laskee etäisyyden km kahden koordinaattipisteen välillä."""
    R = 6371.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2.0)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2.0)**2
    return 2 * R * math.asin(math.sqrt(a))

distances = []
for _, row in tqdm(df_geocoded.iterrows(), total=df_geocoded.shape[0], desc="Calculating distances"):
    lat_c, lon_c = row["lat"], row["lon"]
    # Laske min etäisyys asemille
    min_dist_km = None
    for station_coords in STATIONS.values():
        km = haversine(lat_c, lon_c, station_coords["lat"], station_coords["lon"])
        if min_dist_km is None or km < min_dist_km:
            min_dist_km = km
    walk_min = round((min_dist_km / WALKING_SPEED_KMH) * 60) if min_dist_km is not None else None
    distances.append(walk_min)

df_geocoded["travel_min"] = distances

print("\nYritykset ja niiden kävelyaika lähimpään asemaan:")
print(df_geocoded[["company_name", "full_address", "travel_min"]])

# --- 10. SUODATA ne, joiden kävelyaika ≤ 60 minuuttia ---
MAX_MIN = 60
df_nearby = df_geocoded[df_geocoded["travel_min"] <= MAX_MIN].copy()
print(f"\n{len(df_nearby)} yritystä alle {MAX_MIN} min kävelymatkan päässä asemalta:")
print(df_nearby[["company_name", "full_address", "travel_min"]])

# --- 11. Tallenna lopputulos Exceliin (kaikki sarakkeet) ---
OUTPUT_FILE = "mansala_nearby_all_columns.xlsx"
df_nearby.to_excel(OUTPUT_FILE, index=False)
print(f"\nTulokset (kaikki alkuperäiset sarakkeet + lat/lon + travel_min) tallennettu tiedostoon {OUTPUT_FILE}")

# --- 12. TARKISTA “Recticel Insulation Oy” ---
print("\n--- Tarkistetaan, löydetäänkö 'Recticel Insulation Oy' ja sen tiedot: ---")
mask_recticel = df_nearby["company_name"].str.contains("Recticel Insulation Oy", case=False, na=False)
if mask_recticel.any():
    print("Recticel löytyy lopputuloksesta:")
    print(df_nearby.loc[mask_recticel].T)  # transpoosi, jotta näet sarakkeet riveinä
else:
    print("Recticel ei löydy ryhmästä alle 60 min päässä asemalta.")
    # Näytetään silti kaikki geokoodatut tiedot Recticelistä, jos se oli alkuperäisessä datassa
    mask_rect_all = df_geocoded["company_name"].str.contains("Recticel Insulation Oy", case=False, na=False)
    if mask_rect_all.any():
        print("\nRecticel on geokoodatuissa, mutta sen kävelyaika ylittää 60 min:")
        print(df_geocoded.loc[mask_rect_all].T)
    else:
        print("\nRecticel ei ole geokoodatuissa riveissä lainkaan.")


Haetaan yritykset, joilla postinumero = 04600…
Yrityksiä, joilla osoiterivi 04600: 100 riviä

Esimerkki riveistä osoitetiedoilla (max 5 riviä):
                          company_name          street postcode postOffice
0  Asunto Oy Askolan Riihipellontie 16  Riihipellontie    07500     ASKOLA
1             Asunto Oy Ojasillantie 4    OJASILLANTIE    07230     ASKOLA
2     Asunto-osakeyhtiö Ojasillantie 2    Ojasillantie    07230     ASKOLA
3  Asunto-osakeyhtiö Riihipellontie 12  Riihipellontie    07500     ASKOLA
4               Asunto Oy Haukivalkama      Keskuskatu    04600   MÄNTSÄLÄ

Asunto- ja Kiinteistö-yhtiöt poistettu, jäljellä: 15 riviä

Jäljelle katuosoitteiden mukaan: 15 riviä

Geokoodataan Mäntsälän osoitteet (vain ne, joilla katu löytyy)…


Geocoding:   0%|          | 0/15 [00:00<?, ?it/s]

Geokoodattuja rivejä Mäntsälässä: 13


Calculating distances:   0%|          | 0/13 [00:00<?, ?it/s]


Yritykset ja niiden kävelyaika lähimpään asemaan:
                                         company_name  \
12                                    Nivos Verkot Oy   
13                         OP Koti Ylä-Uusimaa Oy LKV   
37                    Kommandiittiyhtiö Aimo Kiikkilä   
41                               Fennica-Huonekalu Oy   
43  Mäntsälän K-tavaratalo Ostopörssi, Haarala R &...   
52                    Kuljetusliike Teuvo Saarinen Oy   
53                                  Mäntsälän Saha Oy   
77           Mäntsälän Palvelutaksi Kommandiittiyhtiö   
80                 Mäntsälän Autopurkaamo Avoin yhtiö   
81                          Mairuen Kone ja Pultti Oy   
87                                       Lomasalmi Oy   
95                            J. Pajun Autotarvike Ky   
97                                    Marila Infra Oy   

                                full_address  travel_min  
12         SEPÄNTIE 04600 MÄNTSÄLÄ, Mäntsälä          11  
13       Keskuskatu 04600 MÄNTSÄ

In [ ]:
import requests
import pandas as pd

# Hae “Recticel Insulation Oy” nimen perusteella
BASE_URL = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL = f"{BASE_URL}/companies"
HEADERS  = {"Accept": "application/json"}

params = {
    "name": "Recticel Insulation Oy",
    "maxResults": 5,
    "resultsFrom": 0
}

r = requests.get(COMP_URL, params=params, headers=HEADERS, timeout=15)
r.raise_for_status()
data = r.json()
companies = data.get("companies", [])

if not companies:
    print("Yritystä ei löytynyt.")
else:
    company = companies[0]
    # Näytä koko addresses‐lista
    print("Raw 'addresses'-kenttä:")
    from pprint import pprint
    pprint(company.get("addresses", []))


Raw 'addresses'-kenttä:
[{'buildingNumber': '2',
  'postCode': '04600',
  'postOffices': [{'city': 'MÄNTSÄLÄ',
                   'languageCode': '1',
                   'municipalityCode': '505'},
                  {'city': 'MÄNTSÄLÄ',
                   'languageCode': '2',
                   'municipalityCode': '505'}],
  'registrationDate': '2019-12-02',
  'source': '0',
  'street': 'Gneissitie',
  'type': 2}]


In [ ]:
import requests
import pandas as pd

BASE_URL = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL = f"{BASE_URL}/companies"
HEADERS  = {"Accept": "application/json"}

# Vaihda tämä oikeaksi Business ID:ksi, kun tiedät sen YTJ‐palvelusta:
business_id = "2816195-7"

params = {
    "name": "Recticel Insulation Oy",
    "maxResults": 1,
    "resultsFrom": 0
}

r = requests.get(COMP_URL, params=params, headers=HEADERS, timeout=15)
r.raise_for_status()
data = r.json()
companies = data.get("companies", [])

if not companies:
    print(f"Yritystä Business ID:llä {business_id} ei löytynyt.")
else:
    company = companies[0]
    df = pd.json_normalize(company)
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", 200)
    print("Yrityksen tiedot (Business ID -hautuloksesta):")
    print(df.T)  # Tulostetaan kaikki kentät riveittäin

    # Erityisesti osoitelista (addresses) nähdään näin:
    print("\nOsoitelista (addresses):")
    from pprint import pprint
    pprint(company.get("addresses", []))


Yrityksen tiedot (Business ID -hautuloksesta):
                                                                                   0
names                              [{'name': 'Recticel Insulation Oy', 'type': '1...
companyForms                       [{'type': '16', 'descriptions': [{'languageCod...
companySituations                                                                 []
registeredEntries                  [{'type': '0', 'descriptions': [{'languageCode...
addresses                          [{'type': 2, 'street': 'Gneissitie', 'postCode...
tradeRegisterStatus                                                                1
status                                                                             2
registrationDate                                                          2017-03-01
lastModified                                                     2024-11-15 15:48:27
businessId.value                                                           2816195-7
businessId.registr

In [ ]:
import requests
import pandas as pd

# 1. Määritä Business ID (täsmällinen muoto)
business_id = "2816195-7"

# 2. Lähetä haku API:lle businessId.value-parametrilla
BASE_URL = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL = f"{BASE_URL}/companies"
HEADERS  = {"Accept": "application/json"}

params = {
    "businessId.value": business_id,
    "maxResults":       1,
    "resultsFrom":      0
}

r = requests.get(COMP_URL, params=params, headers=HEADERS, timeout=15)
r.raise_for_status()
data = r.json()
companies = data.get("companies", [])

if not companies:
    print(f"Yritystä Business ID:llä {business_id} ei löytynyt.")
else:
    company = companies[0]
    df_rect = pd.json_normalize(company)

    # Näytä kaikki kentät riveinä (transpoosi), jotta näet sarakenimet helposti
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", 200)
    print("Recticel Insulation Oy -tiedot (Business ID -hautuloksesta):")
    print(df_rect.T)

    # Tulosta osoitetiedon rakenne (addresses)
    from pprint import pprint
    print("\nRaw 'addresses'‐lista:")
    pprint(company.get("addresses", []))


Recticel Insulation Oy -tiedot (Business ID -hautuloksesta):
                                                                                   0
names                              [{'name': 'Artjärven Metalli Oy', 'type': '1',...
companyForms                       [{'type': '16', 'descriptions': [{'languageCod...
companySituations                                                                 []
registeredEntries                  [{'type': '1', 'descriptions': [{'languageCode...
addresses                                                                         []
tradeRegisterStatus                                                                4
status                                                                             2
registrationDate                                                          1976-04-23
endDate                                                                   2005-12-19
businessId.value                                                           0100002-9
busi

In [ ]:
# --- Asenna tarvittavat paketit (ajettava kerran) ---
# !pip install pandas requests

import requests
import pandas as pd
import os

# Jos käytät Colabia, liitä Drive:
from google.colab import drive
drive.mount('/content/drive')

# --- ASETUKSET ---
BASE_URL     = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL     = f"{BASE_URL}/companies"
HEADERS      = {"Accept": "application/json"}

POSTCODE    = "04600"
PAGE_SIZE   = 100  # YTJ palauttaa max 100 riviä kerrallaan
results_from = 0

all_companies = []
total_results = None

# Paginoitu haku
while True:
    params = {
        "postCode":    POSTCODE,
        "maxResults":  PAGE_SIZE,
        "resultsFrom": results_from
    }
    r = requests.get(COMP_URL, params=params, headers=HEADERS, timeout=15)
    r.raise_for_status()
    page = r.json()

    companies = page.get("companies", [])
    if not companies:
        break

    # Kerää sivun tulos
    all_companies.extend(companies)
    total_results = page.get("totalResults", len(all_companies))

    print(f"Haettu {len(companies)} riviä (resultsFrom={results_from}). "
          f"Yhteensä löydetty: {total_results}")

    results_from += PAGE_SIZE
    if results_from >= total_results:
        break

print(f"\nYhteensä {len(all_companies)} yritystä haettu postinumerolla {POSTCODE}.")

# --- Tallenna JSON-muodossa Driveen ---
drive_path = "/content/drive/MyDrive/mantsala_companies.json"
pd.Series(all_companies).to_json(drive_path, orient="records", force_ascii=False)
print(f"Tiedot tallennettu JSON-muodossa: {drive_path}")

# --- Vaihtoehtoisesti CSV-muodossa (normalisoitu DataFrame) ---
df_all = pd.json_normalize(all_companies)
csv_path = "/content/drive/MyDrive/mantsala_companies.csv"
df_all.to_csv(csv_path, index=False)
print(f"Tiedot tallennettu CSV-muodossa:   {csv_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Haettu 100 riviä (resultsFrom=0). Yhteensä löydetty: 981
Haettu 100 riviä (resultsFrom=100). Yhteensä löydetty: 981
Haettu 100 riviä (resultsFrom=200). Yhteensä löydetty: 981
Haettu 100 riviä (resultsFrom=300). Yhteensä löydetty: 981
Haettu 100 riviä (resultsFrom=400). Yhteensä löydetty: 981
Haettu 100 riviä (resultsFrom=500). Yhteensä löydetty: 981
Haettu 100 riviä (resultsFrom=600). Yhteensä löydetty: 981
Haettu 100 riviä (resultsFrom=700). Yhteensä löydetty: 981
Haettu 100 riviä (resultsFrom=800). Yhteensä löydetty: 981
Haettu 100 riviä (resultsFrom=900). Yhteensä löydetty: 981

Yhteensä 1000 yritystä haettu postinumerolla 04600.
Tiedot tallennettu JSON-muodossa: /content/drive/MyDrive/mantsala_companies.json
Tiedot tallennettu CSV-muodossa:   /content/drive/MyDrive/mantsala_companies.csv


In [ ]:
# --- Asenna tarvittavat paketit, jos ei jo asennettu ---
# !pip install pandas requests geopy tqdm openpyxl

import requests
import pandas as pd
import numpy as np
import time
from geopy.geocoders import Nominatim
from tqdm.auto import tqdm
import math
import os

# Jos Colabissa: liitä Drive
from google.colab import drive
drive.mount('/content/drive')

# --- ASETUKSET ja muuttujat ---
BASE_URL          = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL          = f"{BASE_URL}/companies"
HEADERS           = {"Accept": "application/json"}
POSTCODE          = "04600"
PAGE_SIZE         = 100            # YTJ palauttaa max 100 riviä kerrallaan
WALKING_SPEED_KMH = 5              # km/h
STATIONS = {                        # Juna-asemat (lat, lon)
    "Mäntsälä": {"lat": 60.6320, "lon": 25.3187},
    "Kerava":   {"lat": 60.4027, "lon": 25.1028},
    "Lahti":    {"lat": 60.9827, "lon": 25.6570},
    "Helsinki": {"lat": 60.1719, "lon": 24.9410}
}

# --- 1. LADATAAN MASSADATA CSV:stä ---
csv_path = "/content/drive/MyDrive/mantsala_companies.csv"
print(f"Ladataan massadata: {csv_path}")
df = pd.read_csv(csv_path)

# --- 2. PURA yrityksen nimi ja osoitetiedot massadatasta (ast.literal_eval tarvittaessa) ---
import ast

def get_company_name(names_cell):
    try:
        L = ast.literal_eval(names_cell)
        return L[0].get("name", None) if isinstance(L, list) and L else None
    except:
        return None

def extract_address_fields(addr_cell):
    try:
        addresses = ast.literal_eval(addr_cell)
    except:
        addresses = []
    if not isinstance(addresses, list) or not addresses:
        return {"street": None, "buildingNumber": None, "postcode": None, "postOffice": None}
    for addr in addresses:
        if not isinstance(addr, dict):
            continue
        street = addr.get("street", None)
        building_num = addr.get("buildingNumber", None)
        postcode = addr.get("postCode", None)
        post_offices = addr.get("postOffices", [])
        post_office = None
        if isinstance(post_offices, list) and post_offices:
            post_office = post_offices[0].get("city", None)
        if street and postcode and post_office:
            return {
                "street": street,
                "buildingNumber": building_num,
                "postcode": postcode,
                "postOffice": post_office
            }
    first = addresses[0]
    if isinstance(first, dict):
        return {
            "street": first.get("street", None),
            "buildingNumber": first.get("buildingNumber", None),
            "postcode": first.get("postCode", None),
            "postOffice": (first.get("postOffices")[0].get("city")
                           if isinstance(first.get("postOffices"), list) and first.get("postOffices") else None)
        }
    return {"street": None, "buildingNumber": None, "postcode": None, "postOffice": None}

df["company_name"]   = df["names"].apply(get_company_name)
addr_df = df["addresses"].apply(extract_address_fields).apply(pd.Series)
df["street"]         = addr_df["street"].fillna("")
df["buildingNumber"] = addr_df["buildingNumber"].fillna("")
df["postcode"]       = addr_df["postcode"].fillna("")
df["postOffice"]     = addr_df["postOffice"].fillna("")

# --- 3. TARKISTA, löytyykö ‘Recticel Insulation Oy’ massadatasta ---
print("\n--- Tarkistetaan, löytyykö 'Recticel Insulation Oy' massadatasta ---")
mask_rect_mass = df["company_name"].str.contains("Recticel Insulation Oy", case=False, na=False)
if mask_rect_mass.any():
    print("Recticel löytyy massadatasta, tässä sen rivi:")
    print(df.loc[mask_rect_mass, ["company_name","street","buildingNumber","postcode","postOffice"]].to_string(index=False))
else:
    print("Recticel EI löytynyt massadatasta.")

# --- 4. HAETAAN RECTICEL INSULATION OY ERIKSEEN BUSINESS ID:LLÄ ---
business_id = "2816195-7"
params_rect = {
    "businessId":  business_id,
    "maxResults":  1,
    "resultsFrom": 0
}
r = requests.get(COMP_URL, params=params_rect, headers=HEADERS, timeout=15)
r.raise_for_status()
data_rect = r.json().get("companies", [])

if not data_rect:
    raise RuntimeError(f"Yritystä Business ID:llä {business_id} ei löytynyt YTJ:stä.")
rect_company = data_rect[0]
rect_df = pd.json_normalize(rect_company)

# --- 5. Tulosta Recticel-tiedot: ensiksi 10 ensimmäistä kenttää ja niiden arvot ---
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)
print("\n--- Recticel Insulation Oy (Business ID -hausta) – 10 ensimmäistä saraketta ja arvot: ---")
first_10_cols = rect_df.columns[:10]
print(rect_df[first_10_cols].T)

# --- 6. VERTAA NÄITÄ KENTTIÄ MASSADATAAN ---
print("\n--- Vertailu Recticelin kentille massadatassa ---")
for col in first_10_cols:
    rect_val = rect_df.iloc[0].get(col, None)
    # Etsitään massadatasta tuon sarakkeen arvo (jos Recticel olisi siellä)
    if mask_rect_mass.any() and col in df.columns:
        mass_val = df.loc[mask_rect_mass, col].iloc[0]
    else:
        mass_val = None
    print(f"{col}:")
    print(f"  Recticel (YTJ hausta): {rect_val}")
    print(f"  Massa‐data       : {mass_val}\n")

# --- 7. POISTA asunto‐ ja kiinteistöyhtiöt massadatasta JA suodata osoitekentän mukaan ---
mask_housing = df["company_name"].str.match(
    r"^(Asunto[\- ]?Oy|Asunto[\- ]?osakeyhtiö|As\.? ?Oy|As\.Oy|AsOy|Kiinteistö[\- ]?Oy|Kiinteistöosakeyhtiö)",
    case=False,
    na=False
)
df = df.loc[~mask_housing].copy()
print(f"\nAsunto‐ ja Kiinteistöyhtiöt poistettu, jäljellä: {len(df)} riviä")

df_with_street = df[df["street"] != ""].copy()
print(f"\nJäljelle katuosoitteiden mukaan: {len(df_with_street)} riviä")

# --- 8. Luo 'full_address' (sis. buildingNumber) ---
df_with_street["full_address"] = df_with_street.apply(
    lambda r: f"{r['street']} {r['buildingNumber']} {r['postcode']} {r['postOffice']}, Mäntsälä",
    axis=1
)
print("\nEsimerkki full_address-kentästä:")
print(df_with_street[["company_name","full_address"]].head())

# --- 9. GEOKOODAA välimuistia käyttäen (Nominatim) ---
CACHE_FILE = "/content/drive/MyDrive/mantsala_geocache.csv"
if os.path.exists(CACHE_FILE):
    cache_df = pd.read_csv(CACHE_FILE)
else:
    cache_df = pd.DataFrame(columns=["full_address","lat","lon"])

geolocator = Nominatim(user_agent="tyomatka_skanneri_demo")

def geocode_with_cache(address):
    row = cache_df.loc[cache_df["full_address"] == address]
    if not row.empty:
        return float(row.iloc[0]["lat"]), float(row.iloc[0]["lon"])
    try:
        loc = geolocator.geocode(address, timeout=10)
        if loc:
            lat, lon = loc.latitude, loc.longitude
        else:
            lat, lon = np.nan, np.nan
    except:
        lat, lon = np.nan, np.nan

    cache_df.loc[len(cache_df)] = [address, lat, lon]
    cache_df.to_csv(CACHE_FILE, index=False)
    time.sleep(1.1)
    return lat, lon

print("\nGeokoodataan Mäntsälän osoitteet (joilla katu löytyy)…")
coords = []
for address in tqdm(df_with_street["full_address"], desc="Geocoding"):
    coords.append(geocode_with_cache(address))

df_with_street[["lat","lon"]] = pd.DataFrame(coords, index=df_with_street.index)
df_geocoded = df_with_street.dropna(subset=["lat","lon"]).copy()
print(f"Geokoodattuja rivejä Mäntsälässä: {len(df_geocoded)}")

# --- 10. Haversine‐etäisyys ja kävelyaika lähimpään asemaan ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2.0)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2.0)**2
    return 2 * R * math.asin(math.sqrt(a))

distances = []
for _, r in tqdm(df_geocoded.iterrows(), total=df_geocoded.shape[0], desc="Calculating distances"):
    lat_c, lon_c = r["lat"], r["lon"]
    min_dist_km = None
    for sta in STATIONS.values():
        km = haversine(lat_c, lon_c, sta["lat"], sta["lon"])
        if min_dist_km is None or km < min_dist_km:
            min_dist_km = km
    walk_min = round((min_dist_km / WALKING_SPEED_KMH) * 60) if min_dist_km is not None else None
    distances.append(walk_min)

df_geocoded["travel_min"] = distances

print("\nYritykset ja niiden kävelyaika lähimpään asemaan (max 5 riviä):")
print(df_geocoded[["company_name","full_address","travel_min"]].head())

# --- 11. SUODATA alle 60 min kävelymatkan päässä ---
MAX_MIN = 60
df_nearby = df_geocoded[df_geocoded["travel_min"] <= MAX_MIN].copy()
print(f"\n{len(df_nearby)} yritystä alle {MAX_MIN} min kävelymatkan päässä asemaa:")
print(df_nearby[["company_name","full_address","travel_min"]].head())

# --- 12. TALLENNA lopputulos (kaikki sarakkeet) Exceliin Driveen ---
OUTPUT_FILE = "/content/drive/MyDrive/mansala_nearby_all_columns.xlsx"
df_nearby.to_excel(OUTPUT_FILE, index=False)
print(f"\nTulokset tallennettu tiedostoon {OUTPUT_FILE}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ladataan massadata: /content/drive/MyDrive/mantsala_companies.csv

--- Tarkistetaan, löytyykö 'Recticel Insulation Oy' massadatasta ---
Recticel EI löytynyt massadatasta.

--- Recticel Insulation Oy (Business ID -hausta) – 10 ensimmäistä saraketta ja arvot: ---
                                                                     0
names                [{'name': 'Recticel Insulation Oy', 'type': '1...
companyForms         [{'type': '16', 'descriptions': [{'languageCod...
companySituations                                                   []
registeredEntries    [{'type': '0', 'descriptions': [{'languageCode...
addresses            [{'type': 2, 'street': 'Gneissitie', 'postCode...
tradeRegisterStatus                                                  1
status                                                               2
registrationDate                         

Geocoding:   0%|          | 0/150 [00:00<?, ?it/s]

Geokoodattuja rivejä Mäntsälässä: 130


Calculating distances:   0%|          | 0/130 [00:00<?, ?it/s]


Yritykset ja niiden kävelyaika lähimpään asemaan (max 5 riviä):
                                         company_name                               full_address  travel_min
12                                    Nivos Verkot Oy        SEPÄNTIE 3 04600 MÄNTSÄLÄ, Mäntsälä          11
13                         OP Koti Ylä-Uusimaa Oy LKV      Keskuskatu 6 04600 MÄNTSÄLÄ, Mäntsälä           5
37                    Kommandiittiyhtiö Aimo Kiikkilä      Keskuskatu 4 04600 MÄNTSÄLÄ, Mäntsälä           5
41                               Fennica-Huonekalu Oy  KENKÄPELLONTIE 1 04600 MÄNTSÄLÄ, Mäntsälä          23
43  Mäntsälän K-tavaratalo Ostopörssi, Haarala R &...   KONKELOPOLKU 13 04600 MÄNTSÄLÄ, Mäntsälä          17

120 yritystä alle 60 min kävelymatkan päässä asemaa:
                                         company_name                               full_address  travel_min
12                                    Nivos Verkot Oy        SEPÄNTIE 3 04600 MÄNTSÄLÄ, Mäntsälä          11
13       

In [ ]:
print("\nEsimerkki sarakkeesta 'names' ensimmäisessä rivissä:")
print(df["names"].iloc[0])



Esimerkki sarakkeesta 'names' ensimmäisessä rivissä:
[{'name': 'Kellonosat Oy', 'type': '1', 'registrationDate': '1958-05-09', 'version': 1, 'source': '1'}]


In [ ]:
print("\nEsimerkki sarakkeesta 'addresses' ensimmäisessä rivissä:")
print(df["addresses"].iloc[0])



Esimerkki sarakkeesta 'addresses' ensimmäisessä rivissä:
[{'type': 1, 'street': 'Keskukatu', 'postCode': '04600', 'postOffices': [{'city': 'MÄNTSÄLÄ', 'languageCode': '1', 'municipalityCode': '505'}, {'city': 'MÄNTSÄLÄ', 'languageCode': '2', 'municipalityCode': '505'}], 'buildingNumber': '4', 'entrance': '', 'apartmentNumber': '', 'apartmentIdSuffix': '', 'co': '', 'registrationDate': '2019-04-05', 'source': '0'}, {'type': 2, 'street': 'Keskuskatu', 'postCode': '04600', 'postOffices': [{'city': 'MÄNTSÄLÄ', 'languageCode': '1', 'municipalityCode': '505'}, {'city': 'MÄNTSÄLÄ', 'languageCode': '2', 'municipalityCode': '505'}], 'postOfficeBox': '', 'buildingNumber': '4', 'entrance': '', 'apartmentNumber': '', 'apartmentIdSuffix': '', 'co': '', 'registrationDate': '2019-04-05', 'source': '0'}]


In [ ]:
print("\n--- Recticel JSON‐haun sarakenimet: ---")
for col in rect_df.columns:
    print(col)



--- Recticel JSON‐haun sarakenimet: ---
names
companyForms
companySituations
registeredEntries
addresses
tradeRegisterStatus
status
registrationDate
lastModified
businessId.value
businessId.registrationDate
businessId.source
euId.value
euId.source
mainBusinessLine.type
mainBusinessLine.descriptions
mainBusinessLine.typeCodeSet
mainBusinessLine.registrationDate
mainBusinessLine.source


In [ ]:
import pandas as pd
import ast
import requests

# 1. Lue massadata CSV:stä
csv_path = "/content/drive/MyDrive/mantsala_companies.csv"
df = pd.read_csv(csv_path)

print("\n--- Massadatan sarakenimet: ---")
for col in df.columns:
    print(col)

# Tulosta yksittäisen rivin esimerkki JSON‐sisällöstä (cellsarjat)
print("\nEsimerkki massadatasta (rivi 0):")
print("names:", df["names"].iloc[0])
print("addresses:", df["addresses"].iloc[0])

# 2. Hae Recticel JSON‐haulla erikseen
BASE_URL = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL = f"{BASE_URL}/companies"
business_id = "2816195-7"
params = {"businessId": business_id, "maxResults": 1, "resultsFrom": 0}
r = requests.get(COMP_URL, params=params, headers={"Accept": "application/json"}, timeout=15)
r.raise_for_status()
rect_company = r.json().get("companies", [])[0]

rect_df = pd.json_normalize(rect_company)

print("\n--- Recticel JSON‐haun sarakenimet: ---")
for col in rect_df.columns:
    print(col)

# Yksi rivi: näytä esimerkkiarvot ensimmäisiltä 10 sarakkeelta
first_10_cols = rect_df.columns[:10]
print("\nEsimerkkiset arvot Recticelin ensimmäisistä 10 sarakkeesta:")
print(rect_df[first_10_cols].T)



--- Massadatan sarakenimet: ---
names
companyForms
companySituations
registeredEntries
addresses
tradeRegisterStatus
status
registrationDate
lastModified
businessId.value
businessId.registrationDate
businessId.source
mainBusinessLine.type
mainBusinessLine.descriptions
mainBusinessLine.typeCodeSet
mainBusinessLine.registrationDate
mainBusinessLine.source
website.url
website.registrationDate
website.source
euId.value
euId.source
endDate

Esimerkki massadatasta (rivi 0):
names: [{'name': 'Asunto Oy Askolan Riihipellontie 16', 'type': '1', 'registrationDate': '1975-06-27', 'version': 1, 'source': '1'}]
addresses: [{'type': 1, 'street': 'Riihipellontie', 'postCode': '07500', 'postOffices': [{'city': 'ASKOLA', 'languageCode': '1', 'municipalityCode': '018'}, {'city': 'ASKOLA', 'languageCode': '2', 'municipalityCode': '018'}], 'buildingNumber': '30', 'entrance': '', 'apartmentNumber': '', 'apartmentIdSuffix': '', 'co': '', 'registrationDate': '2023-10-18', 'source': '0'}, {'type': 2, 'street

In [ ]:
# Varmistetaan, että Colabissa Drive on liitetty ja käsitellään oikeita polkuja

import pandas as pd
import ast
import os

# 1. Mount Google Drive (pakollinen Colabissa)
from google.colab import drive
drive.mount('/content/drive')

# 2. Määritä polku käyttäjän Driveen tallennettuun Recticel-tiedostoon
#    (Varmista, että tiedosto on sijoitettu e.g. MyDrive-kansioon)
rect_input_path = "/content/drive/MyDrive/recticel_insulation.csv"

# 3. Lue CSV Drive-polusta
rect_df_raw = pd.read_csv(rect_input_path)

# 4. Funktiot nimen ja osoitteen purkuun
def get_company_name(names_cell):
    try:
        L = ast.literal_eval(names_cell)
        return L[0].get("name", None) if isinstance(L, list) and L else None
    except:
        return None

def extract_address_fields(addr_cell):
    try:
        addresses = ast.literal_eval(addr_cell)
    except:
        addresses = []
    if not isinstance(addresses, list) or not addresses:
        return {"street": None, "buildingNumber": None, "postcode": None, "postOffice": None}
    for addr in addresses:
        if not isinstance(addr, dict):
            continue
        street = addr.get("street", None)
        building_num = addr.get("buildingNumber", None)
        postcode = addr.get("postCode", None)
        post_offices = addr.get("postOffices", [])
        post_office = None
        if isinstance(post_offices, list) and post_offices:
            post_office = post_offices[0].get("city", None)
        if street and postcode and post_office:
            return {
                "street": street,
                "buildingNumber": building_num,
                "postcode": postcode,
                "postOffice": post_office
            }
    first = addresses[0]
    if isinstance(first, dict):
        return {
            "street": first.get("street", None),
            "buildingNumber": first.get("buildingNumber", None),
            "postcode": first.get("postCode", None),
            "postOffice": (first.get("postOffices")[0].get("city")
                           if isinstance(first.get("postOffices"), list) and first.get("postOffices") else None)
        }
    return {"street": None, "buildingNumber": None, "postcode": None, "postOffice": None}

# 5. Kopioi DataFrame ja sovella muokkaukset
rect_df = rect_df_raw.copy()
rect_df["company_name"] = rect_df["names"].apply(get_company_name)
addr_df = rect_df["addresses"].apply(extract_address_fields).apply(pd.Series)
rect_df["street"] = addr_df["street"].fillna("")
rect_df["buildingNumber"] = addr_df["buildingNumber"].fillna("")
rect_df["postcode"] = addr_df["postcode"].fillna("")
rect_df["postOffice"] = addr_df["postOffice"].fillna("")

# 6. Luo full_address-sarake
rect_df["full_address"] = rect_df.apply(
    lambda r: f"{r['street']} {r['buildingNumber']} {r['postcode']} {r['postOffice']}, Mäntsälä",
    axis=1
)

# 7. Tallenna prosessoitu DataFrame Driveen (CSV-muodossa)
processed_rect_path = "/content/drive/MyDrive/recticel_processed.csv"
rect_df.to_csv(processed_rect_path, index=False, encoding="utf-8-sig")
print(f"Recticel Insulation Oy:n prosessoitu CSV tallennettu:\n  {processed_rect_path}")

# 8. Näytä muokatut kentät vertailun vuoksi
rect_df[["company_name", "street", "buildingNumber", "postcode", "postOffice", "full_address"]]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Recticel Insulation Oy:n prosessoitu CSV tallennettu:
  /content/drive/MyDrive/recticel_processed.csv


,company_name,street,buildingNumber,postcode,postOffice,full_address
0,Recticel Insulation Oy,Gneissitie,2,04600,MÄNTSÄLÄ,"Gneissitie 2 04600 MÄNTSÄLÄ, Mäntsälä"


In [ ]:
import pandas as pd

# Oletetaan, että massadata on jo tallennettuna Driveen
massa_csv = "/content/drive/MyDrive/mantsala_companies.csv"
df_massa = pd.read_csv(massa_csv)

print("Massadatan sarakenimet:")
for col in df_massa.columns:
    print("  ", col)


Massadatan sarakenimet:
   names
   companyForms
   companySituations
   registeredEntries
   addresses
   tradeRegisterStatus
   status
   registrationDate
   lastModified
   businessId.value
   businessId.registrationDate
   businessId.source
   mainBusinessLine.type
   mainBusinessLine.descriptions
   mainBusinessLine.typeCodeSet
   mainBusinessLine.registrationDate
   mainBusinessLine.source
   website.url
   website.registrationDate
   website.source
   euId.value
   euId.source
   endDate


In [ ]:
# Ensimmäinen rivi massadatasta
rivi = df_massa.iloc[0]
print("Massadata (rivi 0):")
print("  company_name:       ", rivi["names"])             # tämä on JSON‐lista merkkijonona
print("  addresses:         ", rivi["addresses"])          # JSON‐lista merkkijonona
print("  street:            ", rivi["street"])
print("  buildingNumber:    ", rivi.get("buildingNumber", None))
print("  postcode:          ", rivi["postcode"])
print("  postOffice:        ", rivi["postOffice"])
print("  lat, lon:          ", rivi.get("lat"), rivi.get("lon"))
print("  travel_min:        ", rivi.get("travel_min"))


Massadata (rivi 0):
  company_name:        [{'name': 'Asunto Oy Askolan Riihipellontie 16', 'type': '1', 'registrationDate': '1975-06-27', 'version': 1, 'source': '1'}]
  addresses:          [{'type': 1, 'street': 'Riihipellontie', 'postCode': '07500', 'postOffices': [{'city': 'ASKOLA', 'languageCode': '1', 'municipalityCode': '018'}, {'city': 'ASKOLA', 'languageCode': '2', 'municipalityCode': '018'}], 'buildingNumber': '30', 'entrance': '', 'apartmentNumber': '', 'apartmentIdSuffix': '', 'co': '', 'registrationDate': '2023-10-18', 'source': '0'}, {'type': 2, 'street': 'Keskuskatu', 'postCode': '04600', 'postOffices': [{'city': 'MÄNTSÄLÄ', 'languageCode': '1', 'municipalityCode': '505'}, {'city': 'MÄNTSÄLÄ', 'languageCode': '2', 'municipalityCode': '505'}], 'buildingNumber': '8', 'co': 'c/o Isännöinti Leivonen Oy', 'registrationDate': '2009-07-22', 'source': '0'}]


KeyError: 'street'

In [ ]:
# Colab-ready: hae Mäntsälän yritykset ja Recticel, yhdistä ja tallenna Driveen

import requests
import pandas as pd
import numpy as np
import ast
import time
from google.colab import drive
import os

# 1. Mount Google Drive
drive.mount('/content/drive')
drive_root = "/content/drive/MyDrive"
os.makedirs(drive_root, exist_ok=True)

# A)	Hae kaikki yritykset, joissa postinumero = 04600 (Mäntsälä), paginoitu haku
BASE_URL = "https://avoindata.prh.fi/opendata-ytj-api/v3"
COMP_URL = f"{BASE_URL}/companies"
HEADERS  = {"Accept": "application/json"}

POSTCODE    = "04600"
PAGE_SIZE   = 100
results_from = 0
all_mantsala = []
total_results = None

print("Haetaan kaikki Mäntsälän yritykset postinumerolla 04600…")
while True:
    params = {
        "postCode":    POSTCODE,
        "maxResults":  PAGE_SIZE,
        "resultsFrom": results_from
    }
    r = requests.get(COMP_URL, params=params, headers=HEADERS, timeout=15)
    r.raise_for_status()
    page = r.json()
    companies = page.get("companies", [])
    if not companies:
        break

    all_mantsala.extend(companies)
    total_results = page.get("totalResults", len(all_mantsala))
    print(f"  Haettu {len(companies)} riviä (resultsFrom={results_from}). Yhteensä: {total_results}")
    results_from += PAGE_SIZE
    if results_from >= total_results:
        break

print(f"Kokoelmassa nyt {len(all_mantsala)} Mäntsälän yritystä.\n")

# Muunna DataFrameksi
df_mantsala = pd.json_normalize(all_mantsala)

# B)	Hae Recticel Insulation Oy erikseen businessId:llä
rect_id = "2816195-7"
params_rect = {"businessId": rect_id, "maxResults": 1, "resultsFrom": 0}
r2 = requests.get(COMP_URL, params=params_rect, headers=HEADERS, timeout=15)
r2.raise_for_status()
rect_list = r2.json().get("companies", [])
assert rect_list, "Recticel Insulation Oy ei löytynyt haulla."

df_rect = pd.json_normalize(rect_list)

# C)	Yhdistä DataFramet
df_combined = pd.concat([df_mantsala, df_rect], ignore_index=True)

# D)	Käsittele yhdistetty DataFrame täsmälleen samalla tavalla:
#	- pura company_name
#	- pura address‐tiedot (street, buildingNumber, postcode, postOffice)
#   - luo full_address
#   - lopuksi tallenna erilliseen CSV:hen Driveen

def get_company_name(names_cell):
    if isinstance(names_cell, list) and names_cell:
        return names_cell[0].get("name")
    return None

def extract_address_fields(addr_list):
    if not isinstance(addr_list, list) or not addr_list:
        return {"street":None,"buildingNumber":None,"postcode":None,"postOffice":None}
    for addr in addr_list:
        if not isinstance(addr, dict):
            continue
        street = addr.get("street")
        building_num = addr.get("buildingNumber")
        postcode = addr.get("postCode")
        post_offices = addr.get("postOffices", [])
        post_office = None
        if isinstance(post_offices, list) and post_offices:
            post_office = post_offices[0].get("city")
        if street and postcode and post_office:
            return {"street":street,"buildingNumber":building_num,"postcode":postcode,"postOffice":post_office}
    first = addr_list[0]
    if isinstance(first, dict):
        return {
            "street": first.get("street"),
            "buildingNumber": first.get("buildingNumber"),
            "postcode": first.get("postCode"),
            "postOffice": (first.get("postOffices")[0].get("city")
                           if isinstance(first.get("postOffices"), list) and first.get("postOffices") else None)
        }
    return {"street":None,"buildingNumber":None,"postcode":None,"postOffice":None}

# Luo sarakkeet käsittelyä varten
df_combined["company_name"]   = df_combined["names"].apply(get_company_name)
addr_fields = df_combined["addresses"].apply(extract_address_fields).apply(pd.Series)
df_combined["street"]         = addr_fields["street"].fillna("")
df_combined["buildingNumber"] = addr_fields["buildingNumber"].fillna("")
df_combined["postcode"]       = addr_fields["postcode"].fillna("")
df_combined["postOffice"]     = addr_fields["postOffice"].fillna("")

# Luo full_address (käyttäen Mäntsälä‐paikkakuntaa, kansallisesti samassa muodossa)
df_combined["full_address"] = df_combined.apply(
    lambda r: f"{r['street']} {r['buildingNumber']} {r['postcode']} {r['postOffice']}, Mäntsälä",
    axis=1
)

# Tallenna yhdistetty ja käsitelty DataFrame CSV:ksi Driveen
output_path = f"{drive_root}/mantsala_plus_recticel.csv"
df_combined.to_csv(output_path, index=False, encoding="utf-8-sig")
print(f"\nYhdistetty ja käsitelty taulukko tallennettu:\n  {output_path}")

# Näytä muutama rivi yhdistetystä datasta
df_combined[["company_name","postcode","postOffice","street","buildingNumber","full_address"]].head(10)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Haetaan kaikki Mäntsälän yritykset postinumerolla 04600…
  Haettu 100 riviä (resultsFrom=0). Yhteensä: 981
  Haettu 100 riviä (resultsFrom=100). Yhteensä: 981
  Haettu 100 riviä (resultsFrom=200). Yhteensä: 981
  Haettu 100 riviä (resultsFrom=300). Yhteensä: 981
  Haettu 100 riviä (resultsFrom=400). Yhteensä: 981
  Haettu 100 riviä (resultsFrom=500). Yhteensä: 981
  Haettu 100 riviä (resultsFrom=600). Yhteensä: 981
  Haettu 100 riviä (resultsFrom=700). Yhteensä: 981
  Haettu 100 riviä (resultsFrom=800). Yhteensä: 981
  Haettu 100 riviä (resultsFrom=900). Yhteensä: 981
Kokoelmassa nyt 1000 Mäntsälän yritystä.


Yhdistetty ja käsitelty taulukko tallennettu:
  /content/drive/MyDrive/mantsala_plus_recticel.csv


,company_name,postcode,postOffice,street,buildingNumber,full_address
0,Asunto Oy Askolan Riihipellontie 16,07500,ASKOLA,Riihipellontie,30,"Riihipellontie 30 07500 ASKOLA, Mäntsälä"
1,Asunto Oy Ojasillantie 4,07230,ASKOLA,OJASILLANTIE,4,"OJASILLANTIE 4 07230 ASKOLA, Mäntsälä"
2,Asunto-osakeyhtiö Ojasillantie 2,07230,ASKOLA,Ojasillantie,2,"Ojasillantie 2 07230 ASKOLA, Mäntsälä"
3,Asunto-osakeyhtiö Riihipellontie 12,07500,ASKOLA,Riihipellontie,26,"Riihipellontie 26 07500 ASKOLA, Mäntsälä"
4,Asunto Oy Haukivalkama,04600,MÄNTSÄLÄ,Keskuskatu,8,"Keskuskatu 8 04600 MÄNTSÄLÄ, Mäntsälä"
5,Asunto Oy Iirislahdentie 11,02630,ESBO,Sinimäentie,10,"Sinimäentie 10 02630 ESBO, Mäntsälä"
6,Kellonosat Oy,04600,MÄNTSÄLÄ,Keskukatu,4,"Keskukatu 4 04600 MÄNTSÄLÄ, Mäntsälä"
7,Asunto Oy Isonnevantie 4,04600,MÄNTSÄLÄ,Keskuskatu,8,"Keskuskatu 8 04600 MÄNTSÄLÄ, Mäntsälä"
8,Asunto Oy Järvenpään Pykälistöntie 13,04420,JÄRVENPÄÄ,Suksitehtaankatu,7,"Suksitehtaankatu 7 04420 JÄRVENPÄÄ, Mäntsälä"
9,Asunto Oy Haavikkopolku 1,04600,MÄNTSÄLÄ,Keskuskatu,8,"Keskuskatu 8 04600 MÄNTSÄLÄ, Mäntsälä"


In [4]:
import nbformat as nbf
from pathlib import Path

NB_IN  = Path('/content/drive/MyDrive/Tyomatka_skanneri.ipynb')
NB_OUT = NB_IN.with_stem(NB_IN.stem + '_clean')

# -------- load ----------
nb = nbf.read(NB_IN, as_version=nbf.NO_CONVERT)

# -------- sanitise -------
for cell in nb.cells:
    w = cell.metadata.get("widgets", None)
    if w is not None and "state" not in w:
        # simplest: drop the widgets sub-dict entirely
        del cell.metadata["widgets"]

# -------- save ----------
nbf.write(nb, NB_OUT)
print("Cleaned notebook saved →", NB_OUT)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Tyomatka_skanneri.ipynb'